### 1- Import test data


In [890]:
from dotenv import load_dotenv
import os
import pprint

import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import pytz
import numpy as np
import time

from pybit.unified_trading import HTTP
from backtesting.test import GOOG
load_dotenv()


#connect to bYBIT

session = HTTP(
    testnet=False,
    demo=True,
    api_key=os.getenv('API_DEMO_KEY'),
    api_secret=os.getenv('API_DEMO_SECRET'),
)

### Bybit methods

In [891]:
from datetime import datetime
import time
def get_account_balance(ticker):
    # Retrieve account balance from Bybit
    response = session.get_wallet_balance(accountType="UNIFIED",coin=ticker)  

    
    if response['retCode'] == 0:
        balance = response['result']['list'][0]['coin'][0]['walletBalance']
        print(f'The balance of {ticker} is: {balance}')
        return balance
    else:
        print("Failed to retrieve account balance:", response['ret_msg'])
        return None
    



def fetch_market_data(symbol, interval, category, start_date=None, end_date=None ):
    #setup time parameters
    if start_date:
        start_date = int(pd.to_datetime(start_date).timestamp()*1000)
 

    if end_date:
        end_date = int(pd.to_datetime(end_date).timestamp()*1000)
    #print(start_ts, end_ts, start_date, end_date)
    #Setup graph
    data = session.get_kline(symbol=symbol, interval=interval, category = category,limit = 1000, start= start_date
    )
    df = pd.DataFrame(data['result']['list'])
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']

    df['Date'] = pd.to_datetime(pd.to_numeric(df['Date']), unit='ms')
    df.set_index('Date', inplace=True)
    melbourne_tz = pytz.timezone('Australia/Melbourne')
    df.index = df.index.tz_localize('UTC').tz_convert(melbourne_tz)
    df.index = df.index.tz_localize(None)

    df = df[::-1]

    #add relevant indicators
    df['RSI'] = ta.rsi(df['Close'].astype(float), length = 14)
    df['Fast_EMA'] = ta.ema(df['Close'].astype(float), length=30)
    df['Slow_EMA'] = ta.ema(df['Close'].astype(float), length=50)

    df['ATR'] = ta.atr(df['High'].astype(float), df['Low'].astype(float),df['Close'].astype(float), length=7)

    bbands = ta.bbands(df['Close'].astype(float), length = 20, std = 2)
    df = df.join(bbands)


    backcandles = 100
    #df.reset_index(inplace=True, drop=True)
    df['EMA_SIGNAL'] = [ema_signal(df, i, backcandles) if i >= backcandles - 1 else 0 for i in range(len(df))]
    df['TOTAL_SIGNAL'] = [total_signal(df, i, backcandles) if i >= backcandles-1 else 0 for i in range(len(df))]


    df['pointpos'] = [
    float(row['Low']) - (float(row['High']) - float(row['Low'])) * 0.5 if row['TOTAL_SIGNAL'] == 1 else  # LONG
    float(row['High']) + (float(row['High']) - float(row['Low'])) * 0.5 if row['TOTAL_SIGNAL'] == -1 else  # SHORT
    None
    for _, row in df.iterrows()

]
    return df

       
    

def ema_signal(df, current_candle, backcandles ):
    df_copy = df.copy()
    start = max(0, current_candle - backcandles) #starts at 0, or whatever candle we can reach . ADD 1 if we want to include current candle right?
    end = current_candle     #add 1 ehre too
    df_new = df_copy.iloc[start:end]

    if all(df_new['Fast_EMA'] > df_new['Slow_EMA']):
        return 1  # Uptrend
    elif all(df_new['Fast_EMA'] < df_new['Slow_EMA']):
        return -1  # Downtrend
    else:
        return 0  
     

def total_signal(df, current_candle, backcandles = 6):

    #if EMA signal is uptrend and we close under bollinger band lower, we return a BUY signal
    if (ema_signal(df, current_candle, backcandles)==1 and df['Close'].astype(float).iloc[current_candle]<=df['BBL_20_2.0'].iloc[current_candle]
    ):
        return 1
    
    
    if (ema_signal(df, current_candle, backcandles)==-1 and df['Close'].astype(float).iloc[current_candle]>=df['BBU_20_2.0'].iloc[current_candle]
    ):
        return -1
    return 0




### Retrieve market data

In [894]:
symbol = 'SOLUSDT'
interval = '5'
category = 'linear'
start_date = '2024-10-01 00:00:00'
end_date = '2023-02-01 00:00:00'


#df = fetch_market_data(symbol,interval, category, start_date, None)

 
#df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].apply(pd.to_numeric)
df = GOOG

#df.reset_index(inplace=True, drop=True)
df['RSI'] = ta.rsi(df['Close'].astype(float), length = 14)
df['Fast_EMA'] = ta.ema(df['Close'].astype(float), length=30)
df['Slow_EMA'] = ta.ema(df['Close'].astype(float), length=50)

df['ATR'] = ta.atr(df['High'].astype(float), df['Low'].astype(float),df['Close'].astype(float), length=7)

bbands = ta.bbands(df['Close'].astype(float), length = 20, std = 2)
df = df.join(bbands)


backcandles = 100
#df.reset_index(inplace=True, drop=True)
df['EMA_SIGNAL'] = [ema_signal(df, i,backcandles) if i >= backcandles - 1 else 0 for i in range(len(df))]
df['TOTAL_SIGNAL'] = [total_signal(df, i,backcandles) if i >= backcandles-1 else 0 for i in range(len(df))]
df['pointpos'] = [
    float(row['Low']) - (float(row['High']) - float(row['Low'])) * 0.5 if row['TOTAL_SIGNAL'] == 1 else  # LONG
    float(row['High']) + (float(row['High']) - float(row['Low'])) * 0.5 if row['TOTAL_SIGNAL'] == -1 else  # SHORT
    None
    for _, row in df.iterrows()

]
display(df)
#display(df[df['TOTAL_SIGNAL']!= 0].tail(30))

#display(df.tail(30))
#print(df.dtypes)
print(len(df))


fig = go.Figure(data = go.Candlestick(x=df.index,
                                      open = df['Open'],
                                      high = df['High'],
                                      low = df['Low'],
                                      close = df['Close']))
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.add_trace(go.Scatter(x=df.index, y=df['Fast_EMA'], line=dict(color='blue'), name='Fast EMA (9)'))
fig.add_trace(go.Scatter(x=df.index, y=df['Slow_EMA'], line=dict(color='red'), name='Slow EMA (21)'))

fig.add_trace(go.Scatter(x=df.index, y=df['BBU_20_2.0'], line=dict(color='green', width = 1), name='Upper Band'))
fig.add_trace(go.Scatter(x=df.index, y=df['BBL_20_2.0'], line=dict(color='orange', width = 1), name='Lower Band'))


fig.add_scatter(x= df.index, y=df['pointpos'], mode='markers', marker=dict(size=5, color = "MediumPurple"), name = "entry")
fig.update_layout(
    width=1200,  
    height=800  
)

fig.show()


,Open,High,Low,Close,Volume,RSI,Fast_EMA,Slow_EMA,ATR,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,EMA_SIGNAL,TOTAL_SIGNAL,pointpos
2004-08-19,100.00,104.06,95.96,100.34,22351900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2004-08-20,101.01,109.08,100.50,108.31,11428600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2004-08-23,110.75,113.48,109.05,109.40,9137200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2004-08-24,111.24,111.60,103.57,104.87,7631300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2004-08-25,104.96,108.00,103.88,106.00,4598900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900,61.477295,766.721568,750.415642,12.581704,744.876910,777.7895,810.702090,8.463110,0.697197,0,0,NaN
2013-02-26,795.00,795.95,784.40,790.13,2202500,61.050100,768.231789,751.973068,12.434318,748.909390,779.7595,810.609610,7.912724,0.668079,0,0,NaN
2013-02-27,794.80,804.75,791.11,799.78,2026100,64.999388,770.267158,753.847849,12.746558,752.490640,782.0645,811.638360,7.563023,0.799513,0,0,NaN
2013-02-28,801.10,806.99,801.03,801.20,2265800,65.552880,772.262825,755.704796,11.955621,756.756835,784.4330,812.109165,7.056349,0.802914,0,0,NaN


2148


In [897]:
df[df['TOTAL_SIGNAL']!= 0].head(20)

,Open,High,Low,Close,Volume,RSI,Fast_EMA,Slow_EMA,ATR,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,EMA_SIGNAL,TOTAL_SIGNAL,pointpos
2005-10-13,302.00,302.00,290.68,297.44,10567700,40.934884,304.825359,300.200128,7.703789,298.559932,309.6060,320.652068,7.135564,-0.050694,1,1,285.020
2005-10-14,299.90,300.23,292.54,296.14,8519100,39.765020,304.265013,300.040907,7.701819,297.552985,309.4030,321.253015,7.659922,-0.059620,1,1,288.695
2006-01-20,438.70,440.03,394.74,399.46,41116700,34.069849,434.350452,418.488927,17.704169,401.232064,442.9555,484.678936,18.838658,-0.021236,1,1,372.095
2006-02-01,389.03,402.00,387.52,401.78,27122500,38.654088,431.708296,421.208279,20.556399,402.239505,444.0395,485.839495,18.827152,-0.005496,1,1,380.280
2006-02-03,393.62,393.90,372.57,381.55,18281800,33.410296,426.319560,418.704766,19.743681,384.831396,438.0950,491.358604,24.316006,-0.030803,1,1,361.905
2006-02-07,382.99,383.70,363.35,367.92,16630200,30.989829,420.064089,415.447053,18.878827,367.935044,429.1180,490.300956,28.515679,-0.000123,1,1,353.175
2008-01-04,679.69,680.96,655.00,657.00,5359800,38.566684,684.473712,671.691915,16.862303,662.356269,693.5640,724.771731,8.999236,-0.085816,1,1,642.020
2008-01-07,653.94,662.28,637.35,649.25,6403400,36.326736,682.201214,670.811840,18.014831,655.207086,690.2635,725.319914,10.157400,-0.084964,1,1,624.885
2008-01-08,653.00,659.96,631.00,631.68,5339100,31.815287,678.941781,669.277258,19.578427,644.570210,686.1040,727.637790,12.107141,-0.155177,1,1,616.520
2008-01-17,620.76,625.74,598.01,600.79,8216800,31.117154,661.827188,660.643027,23.304077,604.061116,666.2455,728.429884,18.667108,-0.026302,1,1,584.145


### Backtesting

In [896]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
def SIGNAL():
    return df['TOTAL_SIGNAL']

print(df['TOTAL_SIGNAL'])


class Bollinger_EMA(Strategy):
    mysize = 0.95
    slcoef = 1.1 # Reduce stop-loss coefficient
    TPSLRatio = 1.5 # Reduce take-profit ratio
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        
        slatr = self.slcoef * self.data.ATR[-1]
        
        TPSLRatio = self.TPSLRatio
  
        if self.signal1[-1]==1 and len(self.trades)==0 :
 
           #long position
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr * TPSLRatio
            #print(f"Long SL={sl1}, TP={tp1}, Entry={self.data.Close[-1]} at {self.data.index[-1]}")
            self.buy(sl=sl1, tp=tp1, size = self.mysize )

            
        elif self.signal1[-1]==-1 and len(self.trades)==0:       
            #Short position
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr * TPSLRatio
            #print(f"Short SL={sl1}, TP={tp1}, Entry={self.data.Close[-1]}")
            self.sell(sl=sl1, tp=tp1, size = self.mysize)
        
bt = Backtest(df, Bollinger_EMA,  cash=1000)
bt.run()


2004-08-19    0
2004-08-20    0
2004-08-23    0
2004-08-24    0
2004-08-25    0
             ..
2013-02-25    0
2013-02-26    0
2013-02-27    0
2013-02-28    0
2013-03-01    0
Name: TOTAL_SIGNAL, Length: 2148, dtype: int64


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     2.60708
Equity Final [$]                    885.49485
Equity Peak [$]                    1088.05376
Return [%]                          -11.45052
Buy & Hold Return [%]               703.45824
Return (Ann.) [%]                    -1.41656
Volatility (Ann.) [%]                 4.13465
CAGR [%]                             -0.97866
Sharpe Ratio                         -0.34261
Sortino Ratio                        -0.46942
Calmar Ratio                         -0.07609
Max. Drawdown [%]                   -18.61663
Avg. Drawdown [%]                    -6.75095
Max. Drawdown Duration     2585 days 00:00:00
Avg. Drawdown Duration      864 days 00:00:00
# Trades                                   16
Win Rate [%]                             25.0
Best Trade [%]                         7.7939
Worst Trade [%]                   

In [666]:
bt.plot()

GridPlot(id='p3352', ...)